In [1]:
import pandas as pd
import os

# --- CONFIGURAZIONE ---
DATA_DIR = '../dataset'
# Usa il file creato con il merge precedente
INPUT_FILE = os.path.join(DATA_DIR, 'VAERS_MERGED_LOGIC_FIX.csv')
OUTPUT_FILE = os.path.join(DATA_DIR, 'VAERS_PRE_TARGET.csv')

print(f"1. Caricamento file: {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE, low_memory=False)

# --- PULIZIA PRE-TARGET ---
print("2. Standardizzazione colonne gravità...")

# Colonne che useremo per il calcolo
target_cols_potential = [
    'DIED', 'L_THREAT', 'HOSPITAL', 'DISABLE',
    'BIRTH_DEFECT', 'ER_VISIT', 'ER_ED_VISIT'
]

# Prendiamo solo quelle che esistono nel file
target_cols = [col for col in target_cols_potential if col in df.columns]

for col in target_cols:
    # Riempie i vuoti con 'N'
    df[col] = df[col].fillna('N')
    # Forza stringa maiuscola senza spazi ("y" -> "Y")
    df[col] = df[col].astype(str).str.upper().str.strip()

# --- CREAZIONE TARGET ---
print("3. Calcolo IS_SEVERE...")

# Crea la colonna Target (1 = Grave, 0 = Non Grave)
df['IS_SEVERE'] = df[target_cols].apply(
    lambda row: 1 if any(val == 'Y' for val in row) else 0,
    axis=1
)

# --- SALVATAGGIO (Senza rimuovere nulla) ---
print(f"4. Salvataggio Dataset Completo in {OUTPUT_FILE}...")
df.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("OPERAZIONE COMPLETATA.")
print(f"Il file contiene {len(df.columns)} colonne (incluse quelle originali + IS_SEVERE).")
print(f"Distribuzione Target:\n{df['IS_SEVERE'].value_counts()}")

1. Caricamento file: data/VAERS_MERGED_LOGIC_FIX.csv...
2. Standardizzazione colonne gravità...
3. Calcolo IS_SEVERE...
4. Salvataggio Dataset Completo in data/VAERS_PRE_TARGET.csv...
------------------------------
OPERAZIONE COMPLETATA.
Il file contiene 45 colonne (incluse quelle originali + IS_SEVERE).
Distribuzione Target:
IS_SEVERE
0    815115
1    197779
Name: count, dtype: int64
